In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression

In [2]:
teams_half = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/TeamsHalf.csv')
batting_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/BattingPost.csv')
fielding_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/FieldingPost.csv')
pitching_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/PitchingPost.csv')
salaries = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/Salaries.csv')
pitching_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/PitchingPost.csv')
teams = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/Teams.csv')
series_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/SeriesPost.csv')

In [53]:
teams.head()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1871,NaN,BS1,BNA,NaN,3,31,NaN,20,10,...,24,0.834,Boston Red Stockings,South End Grounds I,NaN,103,98,BOS,BS1,BS1
1,1871,NaN,CH1,CNA,NaN,2,28,NaN,19,9,...,16,0.829,Chicago White Stockings,Union Base-Ball Grounds,NaN,104,102,CHI,CH1,CH1
2,1871,NaN,CL1,CFC,NaN,8,29,NaN,10,19,...,15,0.818,Cleveland Forest Citys,National Association Grounds,NaN,96,100,CLE,CL1,CL1
3,1871,NaN,FW1,KEK,NaN,7,19,NaN,7,12,...,8,0.803,Fort Wayne Kekiongas,Hamilton Field,NaN,101,107,KEK,FW1,FW1
4,1871,NaN,NY2,NNA,NaN,5,33,NaN,16,17,...,14,0.840,New York Mutuals,Union Grounds (Brooklyn),NaN,90,88,NYU,NY2,NY2


In [55]:
teams.columns 
# Drop divID attendecnee / Ghome / franchID / yearID / DivWin  / WCWin / LgWin / WSWin      

Index(['yearID', 'lgID', 'teamID', 'franchID', 'divID', 'Rank', 'G', 'Ghome',
       'W', 'L', 'DivWin', 'WCWin', 'LgWin', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'CS', 'HBP', 'SF', 'RA', 'ER', 'ERA',
       'CG', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'SOA', 'E', 'DP', 'FP',
       'name', 'park', 'attendance', 'BPF', 'PPF', 'teamIDBR',
       'teamIDlahman45', 'teamIDretro'],
      dtype='object')

In [84]:
wins = teams['W']
loses = teams['L']
df = teams[['yearID','teamID','Rank','W','L']]

In [145]:
# Best TeamS = HOU => The Houston Astros
# LAN => Los Angeles Dodgers
teams = df[df['yearID'] >= 2017].sort_values(by=('W'),ascending=False).reset_index(drop=True)
teams

,yearID,teamID,Rank,W,L
0,2017,LAN,1,104,58
1,2017,CLE,1,102,60
2,2017,HOU,1,101,61
3,2017,WAS,1,97,65
4,2017,ARI,2,93,69
5,2017,BOS,1,93,69
6,2017,CHN,1,92,70
7,2017,NYA,2,91,71
8,2017,COL,3,87,75
9,2017,MIL,2,86,76


In [134]:
# Lets look at the best teams batting,fielding and pitching and compare them to 
# to the worst and then somewhere in the middle 
# we'll then determine what type of alogorithm should best be used 
batting_p.columns

Index(['yearID', 'round', 'playerID', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP'],
      dtype='object')

In [42]:
# Best teams player STATS 
crit1 = batting_p['yearID'] >= 2009
crit2 = batting_p['teamID'] == 'WAS'
#crit3 = batting_p['yearID'] < 2009
critall = crit1 & crit2 
Was_players_batting = batting_p[critall].drop(columns=['SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP', 'lgID', 'teamID', 'round']).reset_index(drop=True)
Was_players_batting

,yearID,playerID,G,AB,R,H,2B,3B,HR,RBI
0,2012,bernaro01,4,2,0,0,0,0,0,0
1,2012,burnese01,2,0,0,0,0,0,0,0
2,2012,clippty01,3,0,0,0,0,0,0,0
3,2012,desmoia01,5,19,2,7,1,0,0,0
4,2012,detwiro01,1,1,0,0,0,0,0,0
5,2012,espinda01,5,15,0,1,0,0,0,0
6,2012,garcich02,2,0,0,0,0,0,0,0
7,2012,gonzagi01,2,3,0,0,0,0,0,0
8,2012,gonzami02,1,0,0,0,0,0,0,0
9,2012,gorzeto01,1,0,0,0,0,0,0,0


In [50]:
## Do the same above but for HOU 
crit1 = batting_p['yearID'] >= 2000
crit3 = batting_p['yearID'] < 2016
crit2 = batting_p['teamID'] == 'HOU'
critall = crit1 & crit2 & crit3
Hou_players_batting = batting_p[critall].drop(columns=['SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP', 'lgID', 'teamID', 'round']).reset_index(drop=True)
Hou_players_batting

,yearID,playerID,G,AB,R,H,2B,3B,HR,RBI
0,2001,aloumo01,3,12,0,2,1,0,0,1
1,2001,ausmubr01,3,8,1,2,0,0,1,2
2,2001,bagweje01,3,7,0,3,0,0,0,0
3,2001,berkmla01,3,12,0,2,0,0,0,0
4,2001,biggicr01,3,12,0,2,0,0,0,0
5,2001,castivi02,3,11,1,3,0,0,1,1
6,2001,cruzne01,2,0,0,0,0,0,0,0
7,2001,doteloc01,2,0,0,0,0,0,0,0
8,2001,eusebto01,1,3,1,2,1,0,0,0
9,2001,hidalri01,3,8,1,1,0,0,0,0


In [18]:
## Do the same above but for DET 
crit1 = batting_p['yearID'] >= 2017
crit2 = batting_p['teamID'] == 'DET'
critall = crit1 & crit2
Det_players_batting = batting_p[critall].drop(columns=['SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP', 'lgID', 'teamID', 'round']).reset_index(drop=True)
Det_players_batting

,yearID,playerID,G,AB,R,H,2B,3B,HR,RBI


In [43]:
# np.mean(teams.W.values) = 81 
## Average team = LAA => Los Angeles Angels
crit1 = batting_p['yearID'] >= 2012
crit2 = batting_p['teamID'] == 'LAA'
critall = crit1 & crit2
Laa_players_batting = batting_p[critall].drop(columns=['SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
      'SF', 'GIDP', 'lgID', 'teamID', 'round']).reset_index(drop=True)
Laa_players_batting

,yearID,playerID,G,AB,R,H,2B,3B,HR,RBI
0,2014,aybarer01,3,11,0,5,1,0,0,0
1,2014,beckhgo01,2,1,0,0,0,0,0,0
2,2014,calhoko01,3,15,1,5,0,0,0,0
3,2014,congeha01,1,1,0,0,0,0,0,0
4,2014,cowgico01,1,0,0,0,0,0,0,0
5,2014,croncj01,3,9,0,1,1,0,0,0
6,2014,freesda01,3,8,1,1,0,0,1,1
7,2014,grillja01,2,0,0,0,0,0,0,0
8,2014,hamiljo03,3,13,0,0,0,0,0,1
9,2014,iannech01,3,10,1,1,0,0,1,1
